### Part A
The fully discretized finite difference equation is:

$$
\frac{T^{n+1} - T^{n}}{\Delta t} = D \left[ 
    \frac{ u_{i + 1,j    ,z    } - 2 u_{i  ,j  ,z  } + u_{i - 1,j    ,z    }}{\Delta X^{2}} + 
    \frac{ u_{i    ,j + 1,z    } - 2 u_{i  ,j  ,z  } + u_{i    ,j - 1,z    }}{\Delta Y^{2}} + 
    \frac{ u_{i    ,j    ,z + 1} - 2 u_{i  ,j  ,z  } + u_{i    ,j    ,z - 1}}{\Delta Z^{2}}
\right] 
$$

### Part C

The indexing scheme that I am using a local level is:

```C++ 
for (int k = 1; k < zSize - 1; ++k) //z-direction
	{
	  for (int j = 1; j < ySize - 1; ++j) //y-direction
	  {
	    for (int i = 1; i < xSize - 1; ++i) //x-direction
	    {
	    	u[(xSize * ySize) * (k    ) + xSize * (j    ) + (i    )];
        }
      }
    }
```
MPI saves memory contiguously first in the x-direction (column-wise), the y-direction (row-wise), and then the z-direction (depth-wise). This schema helps convert the 3D data into a single array.  

This is possible since total index of y increases by a factor of xSize because each row has *x* elements, and the total index of z increases by a factor of xSize * ySize because there are *x \* y* elements in each page of the. 


### Part C
The derived data types that I made for the XY, YZ, and XZ plane are:
```C++
  MPI_Datatype XZ, YZ, XY; 

  //This can be contiguous since the MPI can read the data across the 
  //X row then up the Y rows.
  MPI_Type_contiguous(xSize * ySize, MPI_DOUBLE, &XY);


  //Note that for these datatypes, the count * blocklength should be equal to the
  //number of elements in the desired plane.

  //Creation of the YZ datatype. This will be a vector type that simply 
  //reads up the column by skipping up through the xSize columns.
  MPI_Type_vector(zSize * ySize,     1,         xSize, MPI_DOUBLE, &YZ);

  //Creation of the XZ datatype. This will be a vector type that skips 
  //up the entirity of the XZ plane to get back to the base row. 
  MPI_Type_vector(zSize, xSize, xSize * ySize, MPI_DOUBLE, &XZ);

  //Don't forget to commit your changes to the actual variable.
  MPI_Type_commit(&XY);
  MPI_Type_commit(&YZ);
  MPI_Type_commit(&XZ);
```
